In [38]:
from lusidtools.jupyter_tools import toggle_code

"""Loading Portfolio Level data into SRS

Shows how you can load portfolio level valuation data into the Structured Results Store

Attributes
----------
Structured Result Store
"""

toggle_code("Toggle Docstring")


# Uploading Externally Calculated Data to the Structured Results Store

In this notebook, we will be demonstrating how you can store any external document in the structured results store. Our use case for this notebook will be uploading your own portfolio level performance metrics to LUSID. This will allow you to use your in-house calculated NAV and other metrics.
We will demonstrate this by first creating portfolios and then uploading our own performance metrics to the structured results store.

In [39]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import io
import json
import pytz
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.8665.0,0.5.2624,"{'relation': 'RequestLogs', 'href': 'http://lo..."


In [40]:
srs_api = api_factory.build(lu.StructuredResultDataApi)
aggregation_api = api_factory.build(lu.AggregationApi)
configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)

In [41]:
scope = "srs-portfolio-example"

# Read in and format data

We will first create two portfolios that we will upload performance data to.

In [42]:
portfolio_df = pd.read_csv('data/srs_portfolio_level_example_portfolios.csv')

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required={
        "code": "code",
        "display_name": "display_name",
        "base_currency": "base_currency"
    },
    mapping_optional={
        "created": "$2020-01-01T00:00:00+00:00"
    },
    file_type="portfolios",
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])

,success,failed
0,2,0


Below we can see a table of our two portfolios and their respective externally calculated market value and returns. This is the dataset that we will be uploading to the structured results store.

In [43]:
pf_data = pd.read_csv("data/srs_portfolio_level_data.csv")
pf_data

,portfolio,effective date,market value,daily return gross,daily return net,currency
0,PortfolioA,2020-01-01 00:00:00+00:00,"1,000,000.00",1.09,1.07,USD
1,PortfolioA,2020-02-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD
2,PortfolioA,2020-03-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD
3,PortfolioB,2020-01-01 00:00:00+00:00,"1,000,000.00",1.09,1.07,USD
4,PortfolioB,2020-02-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD
5,PortfolioB,2020-03-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD


## Define data ID

We define our data and upsert it into the structured results store, so that we may retrieve it later.

To maintain LUSID's bitemporal capacities, the data in the structured results store is immutable and if you would like to change it, you must update the version variable for each new/updated dataset.

The data ID is An identifier that uniquely describes an item of structured result data and is needed to upsert data to the structured results store. See https://www.lusid.com/docs/api/#operation/UpsertStructuredResultData for more guidance.

In [44]:

version= "1.02"
srs_ids=[]
portfolios = pf_data.groupby("portfolio")

for pf, pf_df in portfolios:
    
    # generate an SRS ID for the portfolio
    srs_id = lm.StructuredResultDataId(
        source="Client",
        code=pf,
        effective_at=datetime(2020, 1, 1, tzinfo=timezone.utc),
        result_type="Raw")
    
    # append the id to the list of data id's so that we may retrieve them later in the cell below
    srs_ids.append(srs_id) 
    
    # load in the csv file
    s = io.StringIO()
    pf_df.to_csv(s)   
    
    # generate the upsertion request
    request_body={
    pf: lm.UpsertStructuredResultDataRequest(
    id=srs_id,
    data=lm.StructuredResultData(
    document_format="csv",
    version=version,
    name="Data file",
    document=s.getvalue()
        )
      )
    }

    result = srs_api.upsert_structured_result_data(
    scope=scope,
    request_body=request_body)
    display(pd.DataFrame(result.values.items()))

,0,1
0,PortfolioA,2022-02-22 12:35:34.900079+00:00


,0,1
0,PortfolioB,2022-02-22 12:35:35.173204+00:00


## Return Data from store

To test if our data upsertion was successful, we generate a request to see what data we have on our two portfolios. In the below table we can find our externally calculated market value and returns.

In [45]:
portfolios = pf_data['portfolio'].unique()
for i, portfolio in enumerate(portfolios):
    result = srs_api.get_structured_result_data(
    scope=scope, 
    request_body={
        portfolio: srs_ids[i]})
    s = io.StringIO(result.values[portfolio].document)
    doc = pd.read_csv(s)
    display(portfolio)
    display(doc)

'PortfolioA'

,Unnamed: 0,portfolio,effective date,market value,daily return gross,daily return net,currency
0,0,PortfolioA,2020-01-01 00:00:00+00:00,"1,000,000.00",1.09,1.07,USD
1,1,PortfolioA,2020-02-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD
2,2,PortfolioA,2020-03-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD


'PortfolioB'

,Unnamed: 0,portfolio,effective date,market value,daily return gross,daily return net,currency
0,3,PortfolioB,2020-01-01 00:00:00+00:00,"1,000,000.00",1.09,1.07,USD
1,4,PortfolioB,2020-02-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD
2,5,PortfolioB,2020-03-01 00:00:00+00:00,"1,320,000.00",1.09,1.07,USD
